# Building the Asset Universe

In [35]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openbb import obb
import financedatabase as fd
from financetoolkit import Toolkit
from utils.api_keys import API_KEYS

## Defining OpenBB Data Providers

In [ ]:

# OBB Data Providers list:
#-------------------------#
obb_data_providers = [
    "alphavantage",
    "benzinga",
    "biztoc",
    "fmp",
    "fred",
    "nasdaq",
    "polygon",
    "tiingo",
    "yfinance"
]
#-------------------------#
# OBB interval list:
obb_intervals = []
    
#defining the function to get data from OBB
def get_obb_data(ticker, provider, start_date=None, end_date=None, interval=None):
    if provider not in obb_data_providers:
        print(f"Error: provider should be in {obb_data_providers}")
        return None
    try:
        data = obb.equity.price.historical(
            ticker, 
            provider=provider, 
            start_date=start_date, 
            end_date=end_date, 
            interval=interval
        ).to_df()
        return data
    except Exception as e:
        print(f"Error fetching data from {provider}: {e}")
        return None

# Alpha Vantage
alphavantage = obb.equity.price.historical("spy", provider="alphavantage").to_df()
# Benzinga
benzinga = obb.equity.price.historical("spy", provider="benzinga").to_df()
#Biztoc
biztoc = obb.equity.price.historical("spy", provider="biztoc").to_df()
# Financial Modeling Prep
fmp = obb.equity.price.historical("spy", provider="fmp").to_df()
# FRED
fred = obb.equity.price.historical("spy", provider="fred").to_df()
# Nasdaq Data Link
nasdaq = obb.equity.price.historical("spy", provider="nasdaq").to_df()
# Polygon
polygon = obb.equity.price.historical("spy", provider="polygon").to_df()
# Tiingo
tiingo = obb.equity.price.historical("spy", provider="tiingo").to_df()
# Yahoo Finance
yahoo = obb.equity.price.historical("spy", provider="yfinance").to_df()


Testing `OpenBB`to import AAPL data

In [ ]:
apple_data = obb.equity.price.historical("AAPL", provider="yfinance").to_df()

## Testing Finance DataBase Package

In [ ]:
# Load FMP API key from API_KEYS dictionary
fmp_api_key = API_KEYS["fmp"]

### FD Equities

In [ ]:
# Initialize the Equities database
equities = fd.Equities()

# Build Asset Universe based on sector and market cap
# Sector: Information Technology (IT)
# Market Cap: Large Cap, Mid Cap, Small Cap (no micro cap, nano cap, or mega cap)
asset_universe = equities.select(
    sector="Information Technology",
    market_cap=["Large Cap", "Mid Cap", "Small Cap"],
)

In [33]:
# Show number of assets 
asset_universe.groupby("country").size().sort_values(ascending=False)

country
United States           1362
China                    629
Japan                    351
Taiwan                   212
Germany                  211
Canada                   124
United Kingdom           121
France                    96
Israel                    90
India                     77
Australia                 71
Sweden                    69
South Korea               64
Hong Kong                 56
Netherlands               47
Finland                   44
Switzerland               39
Norway                    36
Singapore                 36
Italy                     33
Austria                   33
Denmark                   24
Thailand                  18
Spain                     17
Ireland                   15
Belgium                   15
Brazil                    14
Poland                    12
New Zealand               12
Luxembourg                10
Cayman Islands            10
Czech Republic             8
Cyprus                     5
Turkey                     5
Indone

### FD ETFs